# Tarea 2: Naive Bayes, Linear Models y Neural Networks
**Procesamiento de Lenguaje Natural (CC6205-1 - Otoño 2024)**

## Tarjeta de identificación

**Nombres:** ```Diego Acevedo, Benjamín Aguilar y Luis Montero```

**Fecha límite de entrega 📆:** 06/05.

**Tiempo estimado de dedicación:** 4 horas


## Instrucciones
Bienvenid@s a la segunda tarea en el curso de Natural Language Processing (NLP). Esta tarea tiene como objetivo evaluar los contenidos teóricos de las últimas semanas de clases posteriores a la tarea 1, enfocado principalmente en **Naive Bayes**, **Linear Models** y **Neural Networks**. Si aún no has visto las clases, se recomienda visitar los links de las referencias.

La tarea consta de una una parte práctica con el fín de introducirlos a la programación en Python enfocada en NLP.

* La tarea es en **grupo** (maximo hasta 3 personas).
* La entrega es a través de u-cursos a más tardar el día estipulado arriba. No se aceptan atrasos.
* El formato de entrega es este mismo Jupyter Notebook.
* Al momento de la revisión su código será ejecutado. Por favor verifiquen que su entrega no tenga errores de compilación.
* Completar la tarjeta de identificación. Sin ella no podrá tener nota.

> **Importante:** Esta tarea tiene varios resultados experimentales que pueden variar de acuerdo a sus propias implementaciones. No se busca que los resultados sean exactamente los mismos (por ejemplo, que el accuracy fue el mismo que el que esta en la tarea). Lo importante es que implementen sus funciones, las sepan explicar y que puedan hacer varios experimentos.

## Material de referencia

Diapositivas del curso 📄
    
- [Naive Bayes](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-NB.pdf)
- [Linear Models](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-linear.pdf)
- [Neural Networks](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-neural.pdf)

Videos del curso 📺

- Naive Bayes: [Parte 1](https://www.youtube.com/watch?v=kG9BK9Oy1hU), [Parte 2](https://www.youtube.com/watch?v=Iqte5kKHvzE), [Parte 3](https://www.youtube.com/watch?v=TSJg0_X3Abk)

- Linear Models: [Parte 1](https://www.youtube.com/watch?v=zhBxDsNLZEA), [Parte 2](https://www.youtube.com/watch?v=Fooua_uaWSE), [Parte 3](https://www.youtube.com/watch?v=DqbzhdQa1eQ), [Parte 4](https://www.youtube.com/watch?v=1nfWWXqfAzA)

- Neural Networks: [Parte 1](https://www.youtube.com/watch?v=oHZHA8h2xN0), [Parte 2](https://www.youtube.com/watch?v=2lXank0W6G4), [Parte 3](https://www.youtube.com/watch?v=BUDIi9qItzY), [Parte 4](https://www.youtube.com/watch?v=KKN2Ipy-vGk)


## P0. Cargar un dataset

Importamos algunas librerias que seran utiles.

In [1]:
import pandas as pd
from collections import namedtuple

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\benja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Inicializamos el dataset con particiones de entrenamiento y test. Es un dataset de clasificacion multi-clase de oraciones. Cada oracion puede tener una unica etiqueta ?, + o -. Donde ? indica que la oracion es una pregunta, - que la oracion es negativa y + positiva.

In [2]:
document = namedtuple(
    "document", ("words", "class_")  # avoid python's keyword collision
)

raw_train_set = [
              ['Do you have plenty of time?', '?'],
              ['Does she have enough money?','?'],
              ['Did they have any useful advice?','?'],
              ['What day is today?','?'],
              ["I don't have much time",'-'],
              ["She doesn't have any money",'-'],
              ["They didn't have any advice to offer",'-'],
              ['Have you plenty of time?','?'],
              ['Has she enough money?','?'],
              ['Had they any useful advice?','?'],
              ["I haven't much time",'-'],
              ["She hasn't any money",'-'],
              ["He hadn't any advice to offer",'-'],
              ['How are you?','?'],
              ['How do you make questions in English?','?'],
              ['How long have you lived here?','?'],
              ['How often do you go to the cinema?','?'],
              ['How much is this dress?','?'],
              ['How old are you?','?'],
              ['How many people came to the meeting?','?'],
              ['I’m from France','+'],
              ['I come from the UK','+'],
              ['My phone number is 61709832145','+'],
              ['I work as a tour guide for a local tour company','+'],
              ['I’m not dating anyone','-'],
              ['I live with my wife and children','+'],
              ['I often do morning exercises at 6am','+'],
              ['I run everyday','+'],
              ['She walks very slowly','+'],
              ['They eat a lot of meat daily','+'],
              ['We were in France that day', '+'],
              ['He speaks very fast', '+'],
              ['They told us they came back early', '+'],
              ["I told her I'll be there", '+']
]
tokenized_train_set = [document(words=tuple(word_tokenize(d[0].lower())), class_=d[1]) for d in raw_train_set]
train_set = pd.DataFrame(data=tokenized_train_set)

raw_test_set = [
             ['Do you know who lives here?','?'],
             ['What time is it?','?'],
             ['Can you tell me where she comes from?','?'],
             ['How are you?','?'],
             ['I fill good today', '+'],
             ['There is a lot of history here','+'],
             ['I love programming','+'],
             ['He told us not to make so much noise','+'],
             ['We were asked not to park in front of the house','+'],
             ["I don't have much time",'-'],
             ["She doesn't have any money",'-'],
             ["They didn't have any advice to offer",'-'],
             ['I am not really sure','+']
]
tokenized_test_set = [document(words=tuple(word_tokenize(d[0].lower())), class_=d[1]) for d in raw_test_set]
test_set = pd.DataFrame(data=tokenized_test_set)

Separar en X e y, donde X son oraciones tokenizadas e y es la clase a predecir (o target).

In [3]:
X_train, y_train = train_set.drop(columns="class_"), train_set["class_"]
pd.concat([X_train, y_train], axis=1).sample(10, random_state=934)

,words,class_
21,"(i, come, from, the, uk)",+
2,"(did, they, have, any, useful, advice, ?)",?
10,"(i, have, n't, much, time)",-
33,"(i, told, her, i, 'll, be, there)",+
0,"(do, you, have, plenty, of, time, ?)",?
17,"(how, much, is, this, dress, ?)",?
31,"(he, speaks, very, fast)",+
3,"(what, day, is, today, ?)",?
27,"(i, run, everyday)",+
26,"(i, often, do, morning, exercises, at, 6am)",+


Cantidad de oraciones por clase:

In [4]:
train_set.groupby("class_").count()

,words
class_,
+,13
-,7
?,14


(X, y) para el conjunto de test:

In [5]:
X_test, y_test = test_set.drop(columns="class_"), test_set["class_"]
pd.concat([X_test, y_test], axis=1).sample(10, random_state=934)

,words,class_
4,"(i, fill, good, today)",+
10,"(she, does, n't, have, any, money)",-
5,"(there, is, a, lot, of, history, here)",+
1,"(what, time, is, it, ?)",?
3,"(how, are, you, ?)",?
8,"(we, were, asked, not, to, park, in, front, of...",+
12,"(i, am, not, really, sure)",+
11,"(they, did, n't, have, any, advice, to, offer)",-
2,"(can, you, tell, me, where, she, comes, from, ?)",?
9,"(i, do, n't, have, much, time)",-


Cantidad de oraciones por clase en el conjunto de test:

In [6]:
test_set.groupby("class_").count()

,words
class_,
+,6
-,3
?,4


**Importante:** Hasta el momento hemos creado nuestros conjuntos de train y test. A continuacion ustedes deben implementar tres modelos de clasificacion: Naive-bayes, Linear Model y Neural Network. Aqui va un resumen de cada pregunta y lo que se les pide implementar:

* P1: Naive-bayes
 - Implementar `fit` y `predict`
 - Entrenar
 - Evaluar

* P2: Linear Model
 - Implementar `fit` con *on-line gradient descent* y `predict`
 - Entrenar
 - Evaluar

* P3: Neural Network
 - Implementar un iterador de datos con `datasets` y `dataloaders`
 - Implementar una red neuronal con `pytorch`
 - Implementar loop de entrenamiento de una NN
 - Entrenar
 - Evaluar

## P1. Implementar y evaluar Multinomial Naive-Bayes (2 puntos)

### Clase para clasificador

Cree una clase MyMultinomialNB que en su inicializador reciba el parámetro alpha para su clasficador.

Además, debe implementar los métodos `fit(X, y)`y `predict(X)`.

```python
class MyMultinomialNB():
  def __init__(self, alpha, ...):
    ...

  def fit(self, X, y):
    ...
  
  def predict(self, X):
    ...
    return prediction
```
Para computar el entrenamiento de nuestro clasificador debemos:
- extraer el vocabulario,
- determinar las probabilidades $p(c_j)$ para cada una de las clases posibles,
- determinar las probabilidades $p(w_i|c_j)$ para cada una de las palabras y cada una de las clases.

Para lograr lo anterior, también deberán implementar el método `predict_proba(X)`:

```python
  def predict_proba(self, X):
    return prob
```

**Underflow prevention:** En vez de hacer muchas multiplicaciones de `float`s, reemplácenlas por sumas de logaritmos para prevenir errores de precisión. (Revisen la diapo 26 de las slides).

En su implementación deben considerar la tecnica de *Laplace Smoothing* vista en clases. Especificamente considere que su clase `MyMultinomialNB` reciba un parámetro `alpha` no negativo (es decir, mayor o igual a cero). De tal forma que el la probabilidad de una palabra $w$ dado la clase $c$ viene dado por lo siguiente:

$$
p_\alpha (w|c) = \frac{\#(w, c) + \alpha}{N + \alpha |V|}
$$

donde $\alpha$ es el parámetro `alpha` de *Laplace Smoothing*. Mientras que los otras notaciones corresponden a

* $\#(w, c)$ numero de veces que ocurre la palabra $w$ en documentos con la clase $c$ (pensar en un gran documento $D_c$ que concatena todos los documentos de clase $c$ y luego calcula la frecuencia de la palabra $w$ en $D_c$),
* $N$ es igual a $\sum \{\#(w', c): w' \in V\}$ donde $V$ es el vocabulario,
* $|V|$ tamaño del vocabulario.

### Implementación (1.5 pts.)

Escriba aquí la implementación de la clase `MyMultinomialNB`.

In [13]:
import numpy as np

class MyMultinomialNB():
    ## Implementar aquí su clase
    def __init__(self, classes, alpha=1.0):
        self.alpha = alpha
        self.classes = classes
        self.term_prob = {}
        self.prior_prob = {}
        self.vocabulary = []

    def fit(self, X, y):
        """Ajusta el modelo a partir de datos de entrenamiento

        Args:
          X: Serie de pandas con documentos
          y: Serie de pandas con clases ("class_") de los documentos

        Returns:
          None
        """
        df = pd.concat([X, y], axis=1)
        corpus_size_class = {}
        word_per_class = {}

        # Primer loop para precalcular valores útiles
        for cj in self.classes:
            df_cj = df[df["class_"] == cj] # Se filtra por clase
            corpus_size_class[cj] = 0
            word_per_class[cj] = {}
            for phrase in df_cj["words"]:
                for word in phrase:
                    # Se añade la palabra al vocabulario si no está
                    if word not in self.vocabulary:
                        self.vocabulary.append(word)
                    corpus_size_class[cj] += 1
                    # Se cuenta la aparición de la palabra en la clase
                    if word_per_class[cj].get(word):
                        word_per_class[cj][word] += 1
                    else:
                        word_per_class[cj][word] = 1

        # Segundo loop para calcular probabilidades
        for cj in self.classes:
            df_cj = df[df["class_"] == cj]
            # Probabilidad prior, cantidad de documentos de la clase c_j / documentos totales
            self.prior_prob[cj] = len(df_cj) / len(df)
            self.term_prob[cj] = {}
            for word in self.vocabulary:
                n_k = word_per_class[cj][word] if word_per_class[cj].get(word) else 0
                # Probabilidad de la palabra en un documento de clase c_j
                self.term_prob[cj][word] = (n_k + self.alpha) / (corpus_size_class[cj] + self.alpha * len(self.vocabulary))

    def predict(self, X):
        """Predice las clases más probables de una serie de documentos

        Args:
          X: Serie de pandas con documentos

        Returns:
          Serie de pandas con las clase de cada documento de X
        """

        pred = []

        for phrase in X["words"]:
            first = True
            pred_class = None, -np.inf
            for cj in self.classes:
                prob = np.log(self.prior_prob[cj])
                for word in phrase:
                    if word in self.vocabulary:
                        prob += np.log(self.term_prob[cj][word])
                _, curr_prob = pred_class
                if prob > curr_prob or first:
                    first = False
                    pred_class = cj, prob
            max_class, _ = pred_class
            pred.append(max_class)

        return pred

### Entrenamiento (0.2 pts.)
A continuación, inicialicen y entrenen (ajusten) su clasificador con los datos de entrenamiento.

In [14]:
nb_model = MyMultinomialNB(["+", "-", "?"], alpha=0.1)
nb_model.fit(X_train, y_train)

Pruébenlo utilizando el método `predict()` que implementaron.

In [15]:
from sklearn.metrics import classification_report

In [16]:
# Predict train-set
y_pred = nb_model.predict(X_train)
print(y_pred)

['?', '?', '?', '?', '-', '-', '-', '?', '?', '?', '-', '-', '-', '?', '?', '?', '?', '?', '?', '?', '+', '+', '+', '+', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+']


In [17]:
# Metricas en el conjunto de train
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           +       1.00      1.00      1.00        13
           -       1.00      1.00      1.00         7
           ?       1.00      1.00      1.00        14

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



### Evaluación (0.3 pts.)

Ahora probarán el funcionamiento de su clasificador con un conjunto de test.  Habiendo entrenado su clasificador, clasifiquen los documentos del conjunto de prueba `test_set` usando el método `predict`.


In [18]:
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           +       1.00      0.67      0.80         6
           -       0.60      1.00      0.75         3
           ?       1.00      1.00      1.00         4

    accuracy                           0.85        13
   macro avg       0.87      0.89      0.85        13
weighted avg       0.91      0.85      0.85        13



Comenten sus resultados. Estudien que ocurre para alpha=0, 1 y L donde L es un numero muy grande.

A partir de los datos de test, se puede observar que el modelo es capaz de predecir con gran precisión las frases de interrogación,
pero le cuesta diferenciar entre frases positivas y negativas. Esto puede ser porque para las interrogaciones, siempre debe haber un
signo de interrogación, por lo que si una frase tiene un signo ? se le asigna una mayor probabilidad de pertenecer a esa clase,
mientras que para las negaciones se tienen varios casos de falsos positivos, probablemente porque hay keywords muy comunes en frases
negativas, como not, que pueden estar presentes en una frase positiva, como "he told us not to make so much noise". De esta forma, el
modelo aprendió que not es una frase que muy probablemente es parte de una frase negativa, lo que es cierto, pero es incapaz de examinar
con mayor detalle el contexto de la oración para determinar si efectivamente es una negación o no.

En el caso de usar alpha=0, se tiene el caso donde no se usa suavizado, por lo que se tienen palabras con probabilidad 0, causando que
toda la expresión se vaya a 0. Este valor de alpha dio peores resultados que antes (donde se usó un valor de 0.1). Para el caso de
alpha=1 se tiene la situación del Add-1, donde todas las palabras tienen al menos una aparición en cada clase. No hay mucha variación en
comparación con el modelo con alpha=0.1, pero si se perdió precisión con respecto a la predicción de oraciones de interrogación. Para el
caso de alpha=L, se tiene que al ser L un valor muy grande, el valor real del conteo es depreciable, por lo que todas las palabras
aparecerían de forma equiprobable en el corpus. Esto hace que la predicción solo se realice a partir de la probabilidad prior. Esto
genera un mal rendimiento en el modelo.

## P2. Implementar y evaluar Linear Models (2 puntos)

### Clase para clasificador

Cree una clase MyLinearModel para su clasficador. Debe implementar los métodos `fit(X, y, learning_rate, epochs)`y `predict(X)`.

```python
class MyLinearModel():
  def __init__(self, ...):
    ...

  def fit(self, X, y, learning_rate, epochs):
    ...
  
  def predict(self, X):
    ...
    return prediction
```

El modelo lineal que debe implementar viene dado por:
$$
\vec{\hat{y}} = \text{softmax}(\vec{x} \cdot W + \vec{b})\\
\vec{\hat{y}}_{[i]} = \frac{\exp{z_i}}{\sum_{j} \exp{z_j}}\\
z_i = \vec{x} \cdot W_{[:, i]} + \vec{b}_{[i]}
$$
donde $\vec{x}$ es un documento representado con bolsas de palabras (BoW), $W$ es la matriz de pesos y $\vec{b}$ el bias.

El modelo linea debe ajustarlo considerando como objetivo minimizar la cross-entropy loss, es decir:

$$
L_\text{cross-entropy}(\vec{\hat{y}}, \vec{y}) = - \sum_i \vec{y}_{[i]} \log{ \left( \vec{\hat{y}}_{[i]} \right) }
$$

Para representar un documento `(i, am, not, really, sure)` vectorialmente, utilice `CountVectorizer` de sklearn. De esta manera, el documento queda representado como sigue:

|    |   i |   he |   am |   are |   not |   yes |   really |   sure |
|---:|----:|-----:|-----:|------:|------:|------:|---------:|-------:|
|  0 |   1 |    0 |    1 |     0 |     1 |     0 |        1 |      1 |

**Observación:** Si el documento repite palabras entonces tendrá un número mayor a 1. Si el documento no tiene la palabra entonces tiene un 0. Pensar que las palabras `(he, are, yes)` provienen de otros documentos. Recuerde que el `CountVectorizer` se entrena con más de un documento (es decir, un corpus). Aquí debe usar el conjunto de train.

El método `fit(X, y, learning_rate, epochs)` debe ajustar un `CountVectorizer` para representar vectorialmente el documento. Debe guardar el `CountVectorizer` para cuando quiera hacer predicciones. Dentro del método `fit(X, y, learning_rate, epochs)` debe implementar *On-line gradient descent* (visto en clases), es decir, descenso de gradiente usando un data-point por iteración. Su método debe ser capaz de recibir un `learning_rate` para ponderar el gradiente en cada iteración y fijar un número de `epochs`. Luego de entrenar debe guardar los pesos de su modelo lineal, es decir, $(W, \vec{b})$.

En el algoritmo de descenso de gradiente usando un data-point por iteración, o *On-line gradient descent*, debe implementar manualmente las derivadas. Como conoce el modelo lineal y la funcion objetivo, entonces puede calcular manualmente las derivadas. Para ejemplificar, en cada paso del algoritmo de optimizacion debe actualizar los pesos $(W, \vec{b})$ del siguiente modo:

$$
W \leftarrow W - \lambda \nabla_{W} L_\text{cross-entropy}\\
\vec{b} \leftarrow \vec{b} - \lambda \nabla_{\vec{b}} L_\text{cross-entropy}\\
$$
donde $\lambda$ es el parámetro `learning_rate`, $\nabla_{W} L_\text{cross-entropy}$ el gradiente de la Loss con repecto a la matriz de pesos $W$ y $\nabla_{\vec{b}} L_\text{cross-entropy}$ para el bias $\vec{b}$.

Para implementar el algoritmo *On-line gradient descent* les recomendamos (no es obligatorio hacerlo de este modo) definir una función `get_derivative_W(x, y_target, y_pred, n_classes)` que calcule $\nabla_{W} L_\text{cross-entropy}$ y lo mismo con una función `get_derivative_b(y_target, y_pred, n_classes)` que calcule $\nabla_{\vec{b}} L_\text{cross-entropy}$.

Para implementar el método `predict(self, X)` debera usar su `CountVectorizer` definido en `fit(X, y, learning_rate, epochs)` para representar del mismo modo cualquier documento tanto en train como en test.

### Implementación (1.5 pts.)
Implemente un modelo lineal con métodos `fit(X, y)` y `predict(X)`

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from collections import namedtuple

def softmax(x):
    """Calcula la función softmax de un vector de entrada
    
    Args:
        x (numpy.ndarray): Vector de entrada
    
    Returns:
        numpy.ndarray: Resultado de aplicar la función softmax a x
    """
    soft_max_value = np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)
    return soft_max_value

def get_derivative_W(x, y_target, y_pred, n_classes):
    """Calcula el gradiente de la función de pérdida con respecto a los pesos W
    
    Args:
        x (numpy.ndarray): Vector de características
        y_target (numpy.ndarray): Vector de etiquetas de clase reales
        y_pred (numpy.ndarray): Vector de etiquetas de clase predichas
        n_classes (int): Número de clases
    
    Returns:
        numpy.ndarray: Gradiente de la función de pérdida con respecto a los pesos W
    """
    if n_classes == 2:
        # Si es un problema de clasificación binaria
        gradient = np.dot(x.T, (y_pred - y_target))
    else:
        # Si es un problema de clasificación multiclase
        gradient = np.dot(x.T, (y_pred - y_target)) / len(y_target)
    return gradient

def get_derivative_b(y_target, y_pred, n_classes):
    """Calcula el gradiente de la función de pérdida con respecto al sesgo b
    
    Args:
        y_target (numpy.ndarray): Vector de etiquetas de clase reales
        y_pred (numpy.ndarray): Vector de etiquetas de clase predichas
        n_classes (int): Número de clases
    
    Returns:
        numpy.ndarray: Gradiente de la función de pérdida con respecto al sesgo b
    """
    if n_classes == 2:
        # Si es un problema de clasificación binaria
        gradient = np.sum(y_pred - y_target)
    else:
        # Si es un problema de clasificación multiclase
        gradient = np.sum(y_pred - y_target, axis=0) / len(y_target)
    return gradient
    
def get_preds_tests(X, y, linear_layer):
    """Obtiene las predicciones y el ground-truth a partir de los datos de entrada
    
    Args:
        X (numpy.ndarray): Datos de entrada
        y (numpy.ndarray): Etiquetas de clase reales
        linear_layer (dict): Diccionario con los pesos (W) y el sesgo (b) del modelo
    
    Returns:
        numpy.ndarray: Predicciones del modelo
        numpy.ndarray: Ground-truth a partir de las etiquetas de clase reales
    """
    y_pred = softmax(np.dot(X, linear_layer['W']) + linear_layer['b'])
    predictions = np.argmax(y_pred, axis=1)
    return predictions, y

Document = namedtuple("Document", ("words", "class_"))

class MyLinearModel():
    def __init__(self):
        self.W = None
        self.b = None
        self.vectorizer = None

    def fit(self, X, y, learning_rate, epochs, verbose=False):
        """Entrena el modelo a partir de datos de entrenamiento

        Args:
          X: DataFrame de pandas con documentos
          y: Serie de pandas con clases de los documentos
          learning_rate: tasa de aprendizaje del modelo
          epochs: número de épocas de entrenamiento
          verbose: para imprimir mensajes de progreso

        Returns:
          None
        """
        # Preprocesamiento de los datos
        text_data = [' '.join(doc) for doc in X["words"]]
        self.vectorizer = CountVectorizer(tokenizer=lambda x: x, stop_words=None)
        bow_model = self.vectorizer.fit_transform(text_data)
        X_vec = pd.DataFrame(bow_model.toarray())

        # Convertir y_train a lista de Python (si no lo está)
        y_train_list = y.tolist() if isinstance(y, pd.Series) else y

        # Crear un diccionario que mapee cada clase única a un entero
        class_to_int = {c: i for i, c in enumerate(np.unique(y_train_list))}

        # Convertir los valores de y_train_list a enteros utilizando el diccionario
        y_train_int = [class_to_int[c] for c in y_train_list]

        # Inicializar pesos y sesgo
        n_samples, n_features = X_vec.shape
        n_classes = len(np.unique(y_train_list))
        self.W = np.zeros((n_features, n_classes))
        self.b = np.zeros(n_classes)

        # Entrenamiento con descenso de gradiente
        for epoch in range(epochs):
            for i in range(n_samples):
                # Obtener el documento vectorizado y el target
                x = X_vec.iloc[i].values.reshape(1, -1)

                # Inicializar y_target con el número de clases únicas
                y_target = np.zeros((1, n_classes))

                # Asignar 1 a la posición correspondiente al valor de y_train_int
                y_target[0, y_train_int[i]] = 1

                # Forward pass
                z = np.dot(x, self.W) + self.b
                y_pred = softmax(z)

                # Calcular gradientes
                dW = get_derivative_W(x, y_target, y_pred, n_classes)
                db = get_derivative_b(y_target, y_pred, n_classes)

                # Actualizar pesos y sesgo
                self.W -= learning_rate * dW
                self.b -= learning_rate * db

            if verbose:
                print(f"Epoca {epoch} completada!")

            # Calcular precisión después de todas las épocas
            predictions = get_preds_tests(X_vec.values, y_train_int, {'W': self.W, 'b': self.b})
        
            accuracy = np.mean(predictions == np.array(y_train_int))
            print(f"Accuracy: {accuracy}")

    def predict(self, X):
        """Predice las clases más probables de una serie de documentos

        Args:
          X: DataFrame de pandas con documentos

        Returns:
          Serie de pandas con las clase de cada documento de X
        """
        # Preprocesamiento de los datos
        X_vec = self.vectorizer.transform([' '.join(doc) for doc in X["words"]])

        # Realizar la predicción
        y_pred = softmax(X_vec.dot(self.W) + self.b)

        # Obtener la clase con la probabilidad más alta para cada documento
        predictions = np.argmax(y_pred, axis=1)
        label_map = {0: '+', 1: '-', 2: '?'}
        predictions = [label_map[pred] for pred in predictions]

        return predictions

### Entrenamiento (0.2 pts.)
Inicialicen y entrenen su clasificador con los datos de entrenamiento.

In [31]:
linear_model = MyLinearModel()
linear_model.fit(
    X_train, y_train,
    learning_rate=0.02,
    epochs=15,
    verbose=True)

Epoca 0 completada!
Accuracy: 0.6911764705882353
Epoca 1 completada!
Accuracy: 0.6911764705882353
Epoca 2 completada!
Accuracy: 0.6911764705882353
Epoca 3 completada!
Accuracy: 0.6911764705882353
Epoca 4 completada!
Accuracy: 0.7205882352941176
Epoca 5 completada!
Accuracy: 0.7352941176470589
Epoca 6 completada!
Accuracy: 0.8529411764705882
Epoca 7 completada!
Accuracy: 0.8970588235294118
Epoca 8 completada!
Accuracy: 0.9411764705882353
Epoca 9 completada!
Accuracy: 0.9411764705882353
Epoca 10 completada!
Accuracy: 0.9558823529411765
Epoca 11 completada!
Accuracy: 0.9852941176470589
Epoca 12 completada!
Accuracy: 0.9852941176470589
Epoca 13 completada!
Accuracy: 0.9852941176470589
Epoca 14 completada!
Accuracy: 0.9852941176470589


Pruébenlo utilizando el método `predict()` que implementaron.

In [32]:
# Predict train-set
y_pred = linear_model.predict(X_train)
print(y_pred)

['?', '?', '?', '?', '-', '-', '-', '?', '?', '?', '-', '-', '-', '?', '?', '?', '?', '?', '?', '+', '+', '+', '+', '+', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+']


In [33]:
# Metricas en el conjunto de train
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           +       0.93      1.00      0.96        13
           -       1.00      1.00      1.00         7
           ?       1.00      0.93      0.96        14

    accuracy                           0.97        34
   macro avg       0.98      0.98      0.98        34
weighted avg       0.97      0.97      0.97        34



### Evaluación (0.3 pts.)

Ahora probarán el funcionamiento de su clasificador con un conjunto de test.  Habiendo entrenado su clasificador, clasifiquen los documentos del conjunto de prueba `test_set` usando el método `predict`.

In [23]:
y_pred = linear_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           +       0.67      0.67      0.67         6
           -       1.00      1.00      1.00         3
           ?       0.50      0.50      0.50         4

    accuracy                           0.69        13
   macro avg       0.72      0.72      0.72        13
weighted avg       0.69      0.69      0.69        13



Comenten sus resultados. Estudien que ocurre para al menos tres combinaciones de learning rates y epochs, por ejemplo `learning_rate, epochs = (0.02, 15), (0.1, 10), (0.005, 30)`.

### Primer combinación de parámetros (0.02, 15)

**learning_rate = 0.02** 

**epochs = 15**

In [24]:
#Entrenamiento del modelo
linear_model = MyLinearModel()
linear_model.fit(
    X_train, y_train,
    learning_rate=0.02,
    epochs=15,
    verbose=True)

Epoca 0 completada!
Accuracy: 0.6911764705882353
Epoca 1 completada!
Accuracy: 0.6911764705882353
Epoca 2 completada!
Accuracy: 0.6911764705882353
Epoca 3 completada!
Accuracy: 0.6911764705882353
Epoca 4 completada!
Accuracy: 0.7205882352941176
Epoca 5 completada!
Accuracy: 0.7352941176470589
Epoca 6 completada!
Accuracy: 0.8529411764705882
Epoca 7 completada!
Accuracy: 0.8970588235294118
Epoca 8 completada!
Accuracy: 0.9411764705882353
Epoca 9 completada!
Accuracy: 0.9411764705882353
Epoca 10 completada!
Accuracy: 0.9558823529411765
Epoca 11 completada!
Accuracy: 0.9852941176470589
Epoca 12 completada!
Accuracy: 0.9852941176470589
Epoca 13 completada!
Accuracy: 0.9852941176470589
Epoca 14 completada!
Accuracy: 0.9852941176470589


In [25]:
#Evaluación del modelo
y_pred = linear_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           +       0.67      0.67      0.67         6
           -       1.00      1.00      1.00         3
           ?       0.50      0.50      0.50         4

    accuracy                           0.69        13
   macro avg       0.72      0.72      0.72        13
weighted avg       0.69      0.69      0.69        13



### Segunda combinación de parámetros (0.01, 10)

**learning_rate = 0.01** 

**epochs = 10**

In [26]:
#Entrenamiento del modelo
linear_model = MyLinearModel()
linear_model.fit(
    X_train, y_train,
    learning_rate=0.01,
    epochs=10,
    verbose=True)

Epoca 0 completada!
Accuracy: 0.6911764705882353
Epoca 1 completada!
Accuracy: 0.6911764705882353
Epoca 2 completada!
Accuracy: 0.6911764705882353
Epoca 3 completada!
Accuracy: 0.7058823529411765
Epoca 4 completada!
Accuracy: 0.7205882352941176
Epoca 5 completada!
Accuracy: 0.7352941176470589
Epoca 6 completada!
Accuracy: 0.8088235294117647
Epoca 7 completada!
Accuracy: 0.8823529411764706
Epoca 8 completada!
Accuracy: 0.9117647058823529
Epoca 9 completada!
Accuracy: 0.9411764705882353


In [27]:
#Evaluación del modelo
y_pred = linear_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           +       0.67      1.00      0.80         6
           -       1.00      0.67      0.80         3
           ?       1.00      0.50      0.67         4

    accuracy                           0.77        13
   macro avg       0.89      0.72      0.76        13
weighted avg       0.85      0.77      0.76        13



### Tercera combinación de parámetros (0.005, 30)

**learning_rate = 0.005** 

**epochs = 30**

In [28]:
#Entrenamiento del modelo
linear_model = MyLinearModel()
linear_model.fit(
    X_train, y_train,
    learning_rate=0.005,
    epochs=30,
    verbose=True)

Epoca 0 completada!
Accuracy: 0.6911764705882353
Epoca 1 completada!
Accuracy: 0.6911764705882353
Epoca 2 completada!
Accuracy: 0.6911764705882353
Epoca 3 completada!
Accuracy: 0.7205882352941176
Epoca 4 completada!
Accuracy: 0.7205882352941176
Epoca 5 completada!
Accuracy: 0.7352941176470589
Epoca 6 completada!
Accuracy: 0.7794117647058824
Epoca 7 completada!
Accuracy: 0.8235294117647058
Epoca 8 completada!
Accuracy: 0.8970588235294118
Epoca 9 completada!
Accuracy: 0.9117647058823529
Epoca 10 completada!
Accuracy: 0.9117647058823529
Epoca 11 completada!
Accuracy: 0.9558823529411765
Epoca 12 completada!
Accuracy: 0.9705882352941176
Epoca 13 completada!
Accuracy: 0.9852941176470589
Epoca 14 completada!
Accuracy: 0.9852941176470589
Epoca 15 completada!
Accuracy: 0.9852941176470589
Epoca 16 completada!
Accuracy: 0.9852941176470589
Epoca 17 completada!
Accuracy: 0.9852941176470589
Epoca 18 completada!
Accuracy: 0.9852941176470589
Epoca 19 completada!
Accuracy: 0.9852941176470589
Epoca 20 c

In [29]:
#Evaluación del modelo
y_pred = linear_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           +       0.67      0.67      0.67         6
           -       1.00      1.00      1.00         3
           ?       0.50      0.50      0.50         4

    accuracy                           0.69        13
   macro avg       0.72      0.72      0.72        13
weighted avg       0.69      0.69      0.69        13



## Análisis de los resultados en las 3 experimentaciones

Tras el análisis de los resultados experimentales obtenidos, se destacan las siguientes observaciones:

En el primer grupo de hiperparámetros, caracterizado por una tasa de aprendizaje de 0.02 y 15 epochs de entrenamiento, se observa un accuracy en el test de 0.69, un valor que podría considerarse algo bajo para garantizar la fiabilidad del modelo. Es notable que la precisión, el recall y el f1-score para la clase "-" alcanzan valores de 1.0 en todas las métricas evaluadas, mientras que las clases "+" y "?" muestran desequilibrios, generando valores menos precisos.

Para el segundo caso, donde se emplea una tasa de aprendizaje de 0.01 y 10 epochs de entrenamiento, se logra mejorar el accuracy del modelo, obteniendo un valor de 0.77, lo que lo hace más confiable. En contraste con el caso anterior, se observa un equilibrio en las métricas de rendimiento entre las tres clases, lo que indica una mejora en la capacidad del modelo para distinguir entre las diferentes clases.

En cuanto al tercer caso, con una tasa de aprendizaje de 0.005 y 30 epochs de entrenamiento, se obtienen resultados similares al primer conjunto de hiperparámetros, con un accuracy nuevamente de 0.69. De manera consistente, la clase "-" muestra un desempeño perfecto en todas las métricas evaluadas, mientras que las otras dos clases presentan resultados más variados.

En términos generales, los modelos exhiben una tendencia al sesgo, lo que se traduce en una incapacidad para lograr una convergencia estable en los resultados. Es especialmente notable la dificultad de los modelos para distinguir entre las clasificaciones de "+" y "?", en contraste con "-", que se predice con una precisión perfecta, en dos de las 3 experimentaciones.

Además, se observa que pequeñas variaciones en los hiperparámetros provocan un comportamiento errático en el clasificador, lo que resulta en sesgos hacia ciertas clases. De igual manera, resulta interesante notar que con 2 hiperparámetros distintos, como es el caso de la experimentación 1 y 3, se pueden llegar a obtener resultados muy similares

Una posible explicación para este comportamiento errático de accuracy en el total de las experimentaciones podría residir en el tamaño reducido del corpus de entrenamiento y prueba utilizado en este estudio.

## P3. Implementar y evaluar Neural Networks (2 puntos)

### Especificaciones del clasificador

<img src="https://docs.google.com/drawings/d/e/2PACX-1vSXJm5I61m6w0RHTwBL-iMyeFLr2wXBrKNYxdU8Bu1ymuCFPD9dAPsCzPfvIqwSr8uCiYvWMdnGy1if/pub?w=818&h=503" >

En esta última pregunta, ustedes deberánimplementar y evaluar redes neuronales (como la de la figura de arriba). Para esto debera implementar tres secciones principales:

1. Sección iterador,
2. Sección modelo, y
3. Sección loop de entrenamiento.

> **Recomendación:** Para completar esta pregunta puede guiarse del Auxiliar 2 (clase del día 18/04).

*Seccion iterador*

Para ayudarnos a con el entrenamiento y testing, vamos a utilizar las clases `Dataset` y `DataLoader` de `pytorch` ([ver documentación](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)). En esta sección deberá implementar un contenedor para su conjunto de datos usando la clase `Dataset` de `pytorch`. Para esto deberá crear su propia clase `MyDataset` para gestionar los datos. Ésto le permitirá iterar sobre el conjunto mediante el iterador `DataLoader` de `pytorch` y entrenar sin hacer ningún pre-procesamiento extra a los datos.

**Observación:** Si considera por funcionalidad cambiar los parámetros de la clase `MyDataset` puede hacerlo. Asimismo, puede definir otros parámetros para los métodos de su clase.


```python
class MyDataset(Dataset):
    def __init__(self, data, bow_cols):
      ...

    def __len__(self):
      ...

    def __getitem__(self, index):
      ...
      return x_bow, label
```

*Sección modelo*

En esta sección deberán implementar la clase `MyNeuralNetwork` del modulo de `pytorch` llamado `nn.Module` con el proposito de diseñar una red neuronal como la figura de arriba. Para mas detalle sobre las redes ver Clase NLP-Neural.pdf Slide número 8.

**Observación:** La figura de arriba es solo ilustrativa, ustedes pueden variar la dimension input y output de la capa oculta. Sin embargo deben mantener fija la dimension de la entrada y salida de la red. La entrada depende del tamaño del vocabulario. Mientras que la salida depende de la cantidad de clases de su problema de clasificación (en nuestro caso igual a 3).

Es importante que la clase `MyNeuralNetwork` tenga implementadas apropiadamente el `__init__` con las dimensiones y el `forward` con entrada tipo BoW retornando el último estado de la red (output layer). En el `forward` recomendamos utilizar funciones de activación tipo `nn.ReLU`. Sin embargo, no es completamente obligatorio por lo que pueden usar otras.

```python
class MyNeuralNetwork(nn.Module):
    def __init__(self,
                 dim_vocab,
                 num_classes,
                 dim_hidden_input,
                 dim_hidden_output):

        super(MyNeuralNetwork, self).__init__()
        torch.manual_seed(42)
      ...

    def forward(self, xs_bow):
      ...
      return last_state
  ```

*Sección loop de entrenamiento*

En esta sección deberán implementar el loop de entrenamiento de su red neuronal. Para esto, primero deben definir un `criterion`, en nuestro caso `nn.CrossEntropyLoss()` con la libreria de `pytorch`. Sucesivamente debera definir un optimizador, en nuestro caso `optim.SGD` desde el modulo `optim` de `pytorch`.

El loop de entrenamiento debe seguir la siguiente estructura:
```python
for epoch in range(epochs):
  for (xs_bow, labels) in train_loader:
    ...
```

donde `train_loader` proviene del iterador generado en la "sección iterador".

Dentro de "doble for" debera conjugar apropiadamente `opti.zero_grad()`, `loss = criterion(...)`, `loss.backward()` y `opti.step()` con tal de entrenar correctamente su red neuronal. Incluso entrenar, ya que a veces si no se hace de forma correcta entonces tristemente ¡su red no entrena!

> **Recomendación:** Puede guiarse del Auxiliar 2 para implementar el loop de entrenamiento.

### Preparación de la GPU y los datos de train/test

Importar la libreria `pytorch` y `numpy`

In [7]:
import torch
import numpy as np

Verificar que esta usando GPU. Sino, dirígase a **Runtime > Change runtime type** y seleccione la opción **T4 GPU**.

In [8]:
torch.cuda.is_available()

True

In [9]:
test_set['words'][:3]

0                (do, you, know, who, lives, here, ?)
1                             (what, time, is, it, ?)
2    (can, you, tell, me, where, she, comes, from, ?)
Name: words, dtype: object

Preparación de los conjuntos train y test

In [10]:
pd.set_option('display.max_columns', None)
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(tokenizer=lambda x: list(x), preprocessor=lambda x: x, token_pattern=None)

bow_train = pd.DataFrame(
    bow.fit_transform(train_set["words"]).toarray(),
    columns=bow.get_feature_names_out()
)

bow_test = pd.DataFrame(
    bow.transform(test_set["words"]).toarray(),
    columns=bow.get_feature_names_out()
)

bow_label_train = bow_train.astype(float).copy()
bow_label_test = bow_test.astype(float).copy()

map_from_class_to_int = {
    "?": 0,
    "+": 1,
    "-": 2
}

bow_label_train["class_"] = train_set["class_"] # se añade la clase original a la bolsa de palabras de entrenamiento
bow_label_train["int_class_"] = train_set["class_"].apply(lambda x: map_from_class_to_int[x]) # se añade la clase en formato numérico

bow_label_test["class_"] = test_set["class_"] # se añade la clase original a la bolsa de palabras de test
bow_label_test["int_class_"] = test_set["class_"].apply(lambda x: map_from_class_to_int[x]) # se añade la clase en formato numérico

In [12]:
bow_label_test['int_class_'].sample(5, random_state=934)

4     1
10    2
5     1
1     0
3     0
Name: int_class_, dtype: int64

In [13]:
bow_label_train.sample(2, random_state=934)

,'ll,61709832145,6am,?,a,advice,and,any,anyone,are,as,at,back,be,came,children,cinema,come,company,daily,dating,day,did,do,does,dress,early,eat,english,enough,everyday,exercises,fast,for,france,from,go,guide,had,has,have,he,her,here,how,i,in,is,live,lived,local,long,lot,m,make,many,meat,meeting,money,morning,much,my,n't,not,number,of,offer,often,old,people,phone,plenty,questions,run,she,slowly,speaks,that,the,there,they,this,time,to,today,told,tour,uk,us,useful,very,walks,we,were,what,wife,with,work,you,’,class_,int_class_
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,+,1
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,?,0


In [14]:
bow_label_train.shape

(34, 102)

### Implementación (1.7 pts.)

#### Iterador de conjunto de datos
Implemente su clase `MyDataset` para acceder al dataset.

In [16]:
# Se borra columna de clases originales (pues arroja error dado que numpy no maneja strings)

bow_label_train.drop(columns=["class_"], inplace=True)
bow_label_test.drop(columns=["class_"], inplace=True)


In [17]:
from torch.utils.data import Dataset


class MyDataset(Dataset):

  # Implementar aquí su iterador de datos

    def __init__(self, data, bow_cols):
        self.data = data # dataset
        self.bow_cols = bow_cols # columnas de bag-of-words
        pass

    def __len__(self):
        return len(self.data) # tañamo del dataset

    def __getitem__(self, index):
        label = int(self.data.loc[index,"int_class_"]) # obtener la etiqueta (se le asigna un número a la etiqueta)
        x_bow = torch.tensor(self.data.loc[index, self.bow_cols].values, dtype=torch.float32) # 
        return x_bow, label # retorna la bolsa de palabras y con las etiquetas respectivas

Inicializar cada dataloader con sus cotenedor datos para train y test, y número de batches.

In [18]:
from torch.utils.data import DataLoader

# Se deja un batch de tamaño 4 para entrenamiento y test

# Cargador de datos de entrenamiento

train_loader = DataLoader(
    MyDataset(data = bow_label_train, bow_cols = bow_train.columns),
    batch_size = 4, num_workers = 0, shuffle=False)

# Cargador de datos de test
test_loader = DataLoader(
    MyDataset(data = bow_label_test, bow_cols = bow_test.columns),
    batch_size = 4, num_workers = 0, shuffle=False)



In [20]:
# Probamos que el iterador funcione correctamente. Se aprecia que se obtiene la bolsa de palabras y la etiqueta para un ejemplo.

dat = MyDataset(data = bow_label_train, bow_cols = bow_train.columns)   

for file, label in dat:
    print(file, label)
    break

tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]) 0


**Ejemplo de prueba para un batch de entrenamiento**

In [22]:
# Probamos que el cargador de datos funcione correctamente. Se aprecia que se obtiene un batch de tamaño 4 con la bolsa de palabras y las etiquetas respectivas.

batch = next(iter(train_loader))
print(batch)
print(batch[0].shape)


[tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        

#### Modelo

Implemente a continuación su red neuronal

In [23]:
import torch.nn as nn
import torch.optim as optim

In [39]:
import torch.nn as nn
class MyNeuralNetwork(nn.Module):
    def __init__(self,
                 dim_vocab, # Tamaño del vocabulacio
                 num_classes, # Número de clases
                 dim_hidden_input, # Tamaño de la capa oculta entrada
                 dim_hidden_output): # Tamaño de la capa oculta salida
      
        """Inicializa la red neuronal
        
        Los argumentos son: tamaño del vocabulario, número de clases, tamaño de la capa oculta de entrada y tamaño de la capa oculta de salida
        
        Returns:
        None
        """
        super(MyNeuralNetwork, self).__init__()
        torch.manual_seed(934) # Semilla aleatoria para reproducibilidad
    
        self.fc1 = nn.Linear(dim_vocab, dim_hidden_input) # 1ra capa oculta lineal. Tamaño entrada: Dimensión vocabulario (n palabras); Tamaño salida: Dimensión 1ra capa oculta
        self.fc2 = nn.Linear(dim_hidden_input, dim_hidden_output) # 2da capa oculta lineal. Tamaño entrada: Dimensión 1ra capa oculta; Tamaño salida: Dimensión 2da capa oculta
        self.fc3 = nn.Linear(dim_hidden_output, num_classes) # 3ra capa oculta. Tamaño entrada: Dimensión 2da capa oculta, con salida de tamaño numero de classes
        self.relu = nn.ReLU(inplace=False) # Función de activación ReLU
        self.softmax = nn.Softmax(dim=1) # Función de activación Softmax ***


    def forward(self, xs_bow):
      """Calcula la ultima capa mediante las capas intermedias de la red
      Args:
        xs_bow: Tensor

      Returns:
        Tensor con los valores de prediccion
      """
      ## Implementar aquí el forward-pass
      
      first_state = self.fc1(xs_bow)
      first_state = self.relu(first_state)

      hidden_state1 = self.fc2(first_state)
      hidden_state2 = self.relu(hidden_state1)
      
      last_state = self.fc3(hidden_state2)

      output = self.softmax(last_state)  # Se aplica Softmax a la salida para dar una interpretación probabilística al resultados de los outputs***

      return output

Ejemplo de prueba para su modelo NN para un batch de entrenamiento

In [40]:
test = MyNeuralNetwork(
    dim_vocab=len(train_loader.dataset.bow_cols),
    num_classes=3,
    dim_hidden_input=10,
    dim_hidden_output=5).cuda()

batch = next(iter(train_loader))

test(batch[0].cuda())

tensor([[0.2496, 0.5052, 0.2452],
        [0.2453, 0.5140, 0.2407],
        [0.2496, 0.5003, 0.2501],
        [0.2473, 0.4947, 0.2580]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

#### Entrenamiento

Consideren las siguientes funciones que les serán utiles. Si lo desea puede modificarlas a su conveniencia.

In [41]:
def get_loss(net, iterator, criterion, device):
  net.eval()
  total_loss = 0
  num_evals = 0

  with torch.no_grad():
     for xs_bow, labels in iterator:
      xs_bow, labels = xs_bow.to(device), labels.to(device)
      logits = net(xs_bow)
      loss = criterion(logits, labels)
      total_loss += loss.item() * xs_bow.shape[0]
      num_evals += xs_bow.shape[0]

  return total_loss / num_evals


def get_preds_tests_nn(net, iterator, device):
  net.eval()
  preds, tests = [], []
  
  with torch.no_grad():
    for xs_bow, labels in iterator:
        xs_bow, labels = xs_bow.to(device), labels.to(device)
        logits = net(xs_bow)
        soft_probs = nn.Sigmoid()(logits)
        preds += np.argmax(soft_probs.tolist(), axis=1).tolist()
        tests += labels.tolist()

  return np.array(preds), np.array(tests)

A continuación, inicialicen y entrenen su clasificador con los datos de entrenamiento.

In [42]:
import torch.optim as optim

params = {
    "dim_vocab": len(train_loader.dataset.bow_cols),
    "num_classes": 3,
    "dim_hidden_input": 5,
    "dim_hidden_output": 5,
    "learning_rate": 0.4,
    "epochs": 15
}

device = "cuda" if torch.cuda.is_available() else "cpu" # Se dispone de GPU en local. Se asigna el dispositivo correspondiente

# Inicialice su red neuronal
net = MyNeuralNetwork(
    dim_vocab=params["dim_vocab"],
    num_classes=params["num_classes"],
    dim_hidden_input=params["dim_hidden_input"],
    dim_hidden_output=params["dim_hidden_output"]).to(device)

# Definir la Loss = Cross-entropy
criterion = nn.CrossEntropyLoss().to(device)

# Definir el optimizador = SGD: Stochastic-gradient Descent
opti = optim.SGD(net.parameters(), lr = params["learning_rate"])

# Definir el numero de epocas de entrenamiento
epochs = params["epochs"]
import time

## Implementar desde aqui el ciclo de entrenamiento
## para cada epoca en el conjunto de train
for epoch in range(epochs):
  start_time = time.time()
  for (xs_bow, labels) in train_loader:
    opti.zero_grad()
    
    xs_bow, labels = xs_bow.to(device), labels.to(device)
    logits = net(xs_bow)
    loss = criterion(logits, labels)
    loss.backward()
    opti.step()


total_loss = get_loss(net, train_loader, criterion, device)
y_preds, y_tests = get_preds_tests_nn(net, train_loader, device)
acc = (y_preds == y_tests).sum() / y_preds.shape[0]

secs = int(time.time() - start_time)
mins = secs // 60
secs = secs % 60


print("Epoca {} completada! Loss: {} Accuracy: {}".format(epoch, total_loss, acc))
print(f"Epoca {epoch} completada en {mins} minutos, {secs} segundos")



Epoca 14 completada! Loss: 0.7329799497828764 Accuracy: 0.7941176470588235
Epoca 14 completada en 0 minutos, 0 segundos


Pruebe su modelo entrenado con la función `get_preds_tests_nn`.

In [43]:
# Ya no necesitara calcular gradientes para hacer inferencia
from sklearn.metrics import classification_report

for param in net.parameters():
    param.requires_grad = False

# Calcule el la predicción de su modelo y el ground-truth
y_preds, y_tests = get_preds_tests_nn(net, train_loader, device)
print(classification_report(y_tests, y_preds))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90        14
           1       0.76      1.00      0.87        13
           2       0.00      0.00      0.00         7

    accuracy                           0.79        34
   macro avg       0.53      0.67      0.59        34
weighted avg       0.63      0.79      0.70        34



c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Evaluación (0.3 pts.)

Ahora probarán el funcionamiento de su clasificador con un conjunto de test.  Habiendo entrenado su clasificador, clasifiquen los documentos del conjunto de prueba `test_set` usando la función `get_preds_tests_nn`.

In [31]:
y_preds, y_tests = get_preds_tests_nn(net, test_loader, device)
print(classification_report(y_tests, y_preds))

              precision    recall  f1-score   support

           0       0.57      1.00      0.73         4
           1       1.00      1.00      1.00         6
           2       0.00      0.00      0.00         3

    accuracy                           0.77        13
   macro avg       0.52      0.67      0.58        13
weighted avg       0.64      0.77      0.69        13



c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1ra combinación de hiperparámetros
---  

**dim_hidden_input: 4,**  
**dim_hidden_output: 4,**  


**1. Entrenamiento**

In [32]:
params = {
    "dim_vocab": len(train_loader.dataset.bow_cols),
    "num_classes": 3,
    "dim_hidden_input": 4,
    "dim_hidden_output": 4,
    "learning_rate": 0.4,
    "epochs": 15
}

net1 = MyNeuralNetwork(
    dim_vocab=params["dim_vocab"],
    num_classes=params["num_classes"],
    dim_hidden_input=params["dim_hidden_input"],
    dim_hidden_output=params["dim_hidden_output"]).to(device)

for epoch in range(epochs):
  start_time = time.time()
  for (xs_bow, labels) in train_loader:
    opti.zero_grad()
    
    xs_bow, labels = xs_bow.to(device), labels.to(device)
    logits = net1(xs_bow)
    loss = criterion(logits, labels)
    loss.backward()
    opti.step()


total_loss = get_loss(net1, train_loader, criterion, device)
y_preds, y_tests = get_preds_tests_nn(net1, train_loader, device)
acc = (y_preds == y_tests).sum() / y_preds.shape[0]

secs = int(time.time() - start_time)
mins = secs // 60
secs = secs % 60

print(f"Epoca {epoch} completada en {mins} minutos, {secs} segundos")
print(f"Train Loss: {total_loss}, Train Accuracy: {acc}")

Epoca 14 completada en 0 minutos, 0 segundos
Train Loss: 1.0986543262706083, Train Accuracy: 0.4117647058823529


**2. Evaluación del clasificador NN en train y test** 

In [33]:
for param in net.parameters():
    param.requires_grad = False

# Calcule el la predicción de su modelo y el ground-truth
y_preds, y_tests = get_preds_tests_nn(net1, train_loader, device)
print("Reporte de métricas clasificador en set de entrenamiento:\n",classification_report(y_tests, y_preds))

y_preds, y_tests = get_preds_tests_nn(net1, test_loader, device)
print("\nReporte de métricas clasificador en set de validación:\n",classification_report(y_tests, y_preds))

Reporte de métricas clasificador en set de entrenamiento:
               precision    recall  f1-score   support

           0       0.41      1.00      0.58        14
           1       0.00      0.00      0.00        13
           2       0.00      0.00      0.00         7

    accuracy                           0.41        34
   macro avg       0.14      0.33      0.19        34
weighted avg       0.17      0.41      0.24        34


Reporte de métricas clasificador en set de validación:
               precision    recall  f1-score   support

           0       0.31      1.00      0.47         4
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         3

    accuracy                           0.31        13
   macro avg       0.10      0.33      0.16        13
weighted avg       0.09      0.31      0.14        13



c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metr

2da combinación de hiperparámetros  
---  

**dim_hidden_input: 6,**  
**dim_hidden_output: 6,**  


**1. Entrenamiento**

In [34]:
params = {
    "dim_vocab": len(train_loader.dataset.bow_cols),
    "num_classes": 3,
    "dim_hidden_input": 6,
    "dim_hidden_output": 6,
    "learning_rate": 0.4,
    "epochs": 15
}

net2 = MyNeuralNetwork(
    dim_vocab=params["dim_vocab"],
    num_classes=params["num_classes"],
    dim_hidden_input=params["dim_hidden_input"],
    dim_hidden_output=params["dim_hidden_output"]).to(device)

for epoch in range(epochs):
  start_time = time.time()
  for (xs_bow, labels) in train_loader:
    opti.zero_grad()
    
    xs_bow, labels = xs_bow.to(device), labels.to(device)
    logits = net2(xs_bow)
    loss = criterion(logits, labels)
    loss.backward()
    opti.step()


total_loss = get_loss(net2, train_loader, criterion, device)
y_preds, y_tests = get_preds_tests_nn(net2, train_loader, device)
acc = (y_preds == y_tests).sum() / y_preds.shape[0]

secs = int(time.time() - start_time)
mins = secs // 60
secs = secs % 60

print(f"Epoca {epoch} completada en {mins} minutos, {secs} segundos")
print(f"Train Loss: {total_loss}, Train Accuracy: {acc}")

Epoca 14 completada en 0 minutos, 0 segundos
Train Loss: 1.1191212990704704, Train Accuracy: 0.20588235294117646


**2. Evaluación**

In [35]:
for param in net.parameters():
    param.requires_grad = False

# Calcule el la predicción de su modelo y el ground-truth
y_preds, y_tests = get_preds_tests_nn(net2, train_loader, device)
print("Reporte de métricas clasificador en set de entrenamiento:\n",classification_report(y_tests, y_preds))

y_preds, y_tests = get_preds_tests_nn(net2, test_loader, device)
print("\nReporte de métricas clasificador en set de validación:\n",classification_report(y_tests, y_preds))

Reporte de métricas clasificador en set de entrenamiento:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.00      0.00      0.00        13
           2       0.21      1.00      0.34         7

    accuracy                           0.21        34
   macro avg       0.07      0.33      0.11        34
weighted avg       0.04      0.21      0.07        34


Reporte de métricas clasificador en set de validación:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00         6
           2       0.23      1.00      0.38         3

    accuracy                           0.23        13
   macro avg       0.08      0.33      0.12        13
weighted avg       0.05      0.23      0.09        13



c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metr

3ra combinación de hiperparámetros  
---  

**dim_hidden_input: 6,**  
**dim_hidden_output: 4,**  


**1. Entrenamiento**

In [36]:
params = {
    "dim_vocab": len(train_loader.dataset.bow_cols),
    "num_classes": 3,
    "dim_hidden_input": 6,
    "dim_hidden_output": 4,
    "learning_rate": 0.4,
    "epochs": 15
}

net3 = MyNeuralNetwork(
    dim_vocab=params["dim_vocab"],
    num_classes=params["num_classes"],
    dim_hidden_input=params["dim_hidden_input"],
    dim_hidden_output=params["dim_hidden_output"]).to(device)

for epoch in range(epochs):
  start_time = time.time()
  for (xs_bow, labels) in train_loader:
    opti.zero_grad()
    
    xs_bow, labels = xs_bow.to(device), labels.to(device)
    logits = net2(xs_bow)
    loss = criterion(logits, labels)
    loss.backward()
    opti.step()


total_loss = get_loss(net3, train_loader, criterion, device)
y_preds, y_tests = get_preds_tests_nn(net3, train_loader, device)
acc = (y_preds == y_tests).sum() / y_preds.shape[0]

secs = int(time.time() - start_time)
mins = secs // 60
secs = secs % 60

print(f"Epoca {epoch} completada en {mins} minutos, {secs} segundos")
print(f"Train Loss: {total_loss}, Train Accuracy: {acc}")

Epoca 14 completada en 0 minutos, 0 segundos
Train Loss: 1.096057232688455, Train Accuracy: 0.38235294117647056


**2. Evaluación**

In [37]:
for param in net.parameters():
    param.requires_grad = False

# Calcule el la predicción de su modelo y el ground-truth
y_preds, y_tests = get_preds_tests_nn(net3, train_loader, device)
print("Reporte de métricas clasificador en set de entrenamiento:\n",classification_report(y_tests, y_preds))

y_preds, y_tests = get_preds_tests_nn(net3, test_loader, device)
print("\nReporte de métricas clasificador en set de validación:\n",classification_report(y_tests, y_preds))

Reporte de métricas clasificador en set de entrenamiento:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.38      1.00      0.55        13
           2       0.00      0.00      0.00         7

    accuracy                           0.38        34
   macro avg       0.13      0.33      0.18        34
weighted avg       0.15      0.38      0.21        34


Reporte de métricas clasificador en set de validación:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.46      1.00      0.63         6
           2       0.00      0.00      0.00         3

    accuracy                           0.46        13
   macro avg       0.15      0.33      0.21        13
weighted avg       0.21      0.46      0.29        13



c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luian\anaconda3\Lib\site-packages\sklearn\metr

Comenten sus resultados. Estudien que ocurre para al menos tres combinaciones de `(dim_hidden_input, dim_hidden_output)`.

```
Comentar aquí. 

```

**ANÁLISIS DE LOS RESUTLADOS DE LOS 3 EXPERIMENTOS IMPLEMENTADOS**  
====

En primer lugar se puede anotar que el modelo de base (con capas ocultas de tamaño 5 neuronas), arroja en test un accuracy de 0.77 (se debe indicar que se agregó una función softmax a la capa de salida para dar una interpretación probabilística a los vectores de salida). Al testear el modelo en los datos de test, se puede apreciar un recall de 1 para las clases 0 (interrogación) y 1 (clase positiva), pero a costa de la precision. En el caso de la clase negativa, el modelo base no reconoce en los nuevos datos este tipo de texto. Esto se debe a que en entrenamiento no reconoce este tipo de texto. Se puede indicar que el modelo de entrada queda sesgado, pues no reconoce este tipo de oraciones dentro del corpus. De ahí que mientras el F1 para la clase positiva sea 1, este indicador queda indefinido para la clase 

En segundo lugar, al probarse la combinación de 4 capas ocultas de entrada y de salida, el accuracy en test empeora: desciende a 0.41; quedando el modelo sesgado completamente a la clase 0 (interrogación). El recall para esa clase es 1 pero a costa de la precision (0.31), dando un F1 Score de 0.47. Dado que el modelo no asigna ningún ejemplo del test a las clases restantes, precision y recall es cero, quedando indefinido el F1. 

En tercer lugar, al probarse la combinación de 6 capas ocultas de entrada y de salida, nuevamente el accuracy en test empeora: desciende ahora a 0.21; quedando el modelo sesgado completamente a la clase 2 (clase negativa). Nuevamente, el recall para esa clase es 1 pero a costa de la precision (0.23), dando un F1 Score de 0.38. Dado que el modelo no asigna ningún ejemplo del test a las clases restantes, precision y recall es cero, quedando indefinido en ambos casos el F1. 

Por último, al probarse la combinación de 6 capas ocultas de entrada y 4 capas ocultas de salida, el accuracy en test, respecto del experimento anterior mejora, aunque sigue siendo deficiente: desciende a 0.46. En este último experimento, el modelo queda sesgado a la clase 1 (positiva). El recall para esa clase es 1 y la precision es 0.46, dando un F1 Score de 0.63. Dado que el modelo no asigna ningún ejemplo del test a las clases restantes, precision y recall es cero, quedando indefinido el F1.

En síntesis, en función del análisis de los resultados arrojados por el clasificador en testing se puede indicar que presenta problemas de sesgo, no logrando converger a resultados estables: la realizar pequeñas variaciones en los hiperparámetros indicados (capas ocultas de entrada y capas ocultas de salida), el clasificador exhibe un comportamiento errático; siempre sesgándose hacia clases distintas en cada experimiento. Se hipotetiza que el clasificar exhibe este comportamiento por lo reducido del tamaño del copus (pocos documentos de train y de testing). 
 
